In [39]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate
import os

# LLM Model and API Key

In [36]:
# Locate .env file, and load environment variables.
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
else:
    raise FileNotFoundError(".env file not found!")


# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-1.5-flash"

# Load the API key to a variable.
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY is missing! Check your .env file.")


# Basic Query

In [37]:
# Initialize the model.
llm = ChatGoogleGenerativeAI(model=GEMINI_MODEL, google_api_key=GEMINI_API_KEY, temperature=0.3)

In [41]:
restaurant_name = "Pho Binh"
restaurant_city = "Houston"
restaurant_street = "Beechnut"

# restaurant_name = "Biryani Pot"
# restaurant_city = "Houston"
# restaurant_street = "Westheimer"

# restaurant_name = "Chama Gaucha Brazilian Steakhouse"
# restaurant_city = "Houston"
# restaurant_street = "Westheimer"

# Define a query as a string.
# query = f"Give me a summary and review of {restaurant_name} in {restaurant_city} on {restaurant_street}"

query = f"Give me the details of {restaurant_name} in {restaurant_city} on {restaurant_street}"

# Pass the query to the invoke method and print the result.
response = llm.invoke(query)
print(response.content)

I do not have access to real-time information, including restaurant details like menus, hours, or contact information.  To get the details of Pho Binh on Beechnut in Houston, I recommend trying the following:

* **Google Search:** Search "Pho Binh Beechnut Houston" on Google.  This should bring up their Google My Business listing (if they have one), which often includes address, phone number, hours, photos, and sometimes even a menu.
* **Yelp:** Search for "Pho Binh Beechnut Houston" on Yelp.  Yelp provides user reviews, photos, and business information.
* **Other Online Directories:** Check other online directories like TripAdvisor, Zomato, or similar services.
* **Directly Call:** If you can find a phone number through any of the above methods, calling them directly is the most reliable way to get up-to-date information.


Remember to specify "Beechnut" in your search to ensure you find the correct location.


# Templates for Instructions

In [34]:
# Define the format for the template.

format = """You are a world famous restaurant critic.  Answer only questions that would be about restaurants.
If the human asks questions not related to restaurant, remind them that your job is to help them get the details of a restaurnt
Question: {query}
Answer:

    **Your response must be in the following format:**

    **Summary:** [Summary of the restaurant]
    **Moods:** [Moods of the restaurant]
    **Highlight:** [Food quality, Service, Value]
    **Rating:** [Rating]

    For example:
    **Summary:** Chama Gaucha is a high-end churrascaria (Brazilian steakhouse) offering a wide selection of grilled meats carved tableside by gauchos (Brazilian cowboys)
    **Moods:** Vibrant, lively, and celebratory
    **Highlight:** Reviewers consistently praise the "perfectly cooked meats" and the "robust salad bar" with fresh options. 
                The service is frequently described as "impeccable" and "attentive," with staff ensuring glasses are filled and meat preferences are met.
    **Rating:** 4.7

    Do not include any extra text or formatting. 
"""

# Construct the prompt template.
prompt_template = PromptTemplate(
    input_variables=["query"], 
    template = format)

In [31]:
response = (prompt_template | llm).invoke(query)
print(response.content)  

I need more information to answer your question.  My job is to help you get details about restaurants.  Please provide me with the name and location of the restaurant you'd like information on.


<!-- # Combining Chains -->